# Exercises for 2021-03-18

In [ ]:
using LinearAlgebra
using Plots

## Non-negative matrix factorization

Consider the problem of finding the best rank 3 NMF approximation of the following 10-by-5 data matrix.

In [ ]:
A = [0 0 0 1 0;
     0 0 0 0 1;
     0 0 0 0 1;
     1 0 1 0 0;
     1 0 0 0 0;
     0 1 0 0 0;
     1 0 1 1 0;
     0 1 1 0 0;
     0 0 1 1 1;
     0 1 1 0 0]

Most of the methods we have for NMF are iterative (unless some separability condition is satisfied), which means we need an initial guess.
We use an SVD-based initial guess due to Boutsidis and Gallapoulos ([2007 paper](http://scgroup.hpclab.ceid.upatras.gr/faculty/stratis/Papers/HPCLAB020107.pdf)).

In [ ]:
function svd_nmf_init(A, k)
    U, σs, V = svd(A)
    W = zeros(size(A)[1], k)
    H = zeros(k, size(A)[2])
    for j = 1:k
        xp = max.( U[:,j], 0.0)
        xn = max.(-U[:,j], 0.0)
        yp = max.( V[:,j], 0.0)
        yn = max.(-V[:,j], 0.0)
        norm_xp = norm(xp)
        norm_xn = norm(xn)
        norm_yp = norm(yp)
        norm_yn = norm(yn)
        norms_p = norm_xp * norm_yp
        norms_n = norm_xn * norm_yn
        if norms_p > norms_n
            scale = sqrt(σs[j] * norms_p)
            W[:,j] = (scale/norm_xp) * xp
            H[j,:] = (scale/norm_xp) * yp
        else
            scale = sqrt(σs[j] * norms_n)
            W[:,j] = (scale/norm_xn) * xn
            H[j,:] = (scale/norm_yn) * yn
        end
    end
    return W, H
end

### Projective gradient

We will start with the simple projective gradient iteration from the class notes.

In [ ]:
α = 0.1
W, H = svd_nmf_init(A, 3)
resids = zeros(500)
for k = 1:500
    R = A-W*H
    resids[k] = norm(R)
    Wnew = max.(W + α*R*H', 0.0)
    Hnew = max.(H + α*W'*R, 0.0)
    W, H = Wnew, Hnew
end
plot(resids)

In [ ]:
W

In [ ]:
H

In [ ]:
resids[end]

### Lee and Seung

The Lee and Seung iteration takes a while to converge, as we can see from a plot of the residual norm.

In [ ]:
W, H = svd_nmf_init(A, 3)
resids = zeros(500)
for k = 1:500
    resids[k] = norm(A-W*H)
    Wnew = W ./ (W*H*H') .* (A*H')
    Hnew = H ./ (W'*W*H) .* (W'*A)
    W, H = Wnew, Hnew
end
plot(resids)

In [ ]:
W

In [ ]:
H

In [ ]:
A-W*H

In [ ]:
resids[end]

### HALS/RRI (3 points)

Your job is to implement the HALS iteration (also known as RRI), as described in the class notes.  As with the other iterations, show a plot of the residual, and print the final $W$ and $H$ factors.  Do they look similar to the factors computed by either of the previous methods?

### Alternating least squares

#### The NNLS building block

The alternating least squares approach for non-negative matrix factorization requires solving a series of non-negative least squares problem.
For this, it's useful to have a reasonable non-negative least squares problem.  We include below an active set solver for minimizing $\|Ax-b\|^2$ subject to elementwise non-negativity constraints on $A$, essentially identical to the method proposed by [Bro and De Jong](https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/abs/10.1002/%28SICI%291099-128X%28199709/10%2911%3A5%3C393%3A%3AAID-CEM483%3E3.0.CO%3B2-L).

This could be made faster by being a little smarter about the linear algebra at each step, and could potentially be a little more stable by not explicitly forming $A^T A$ and $A^T b$ directly (and instead using the QR factorization).

In [ ]:
function nnls(AtA, Atb, x = [])

  # Set up LS problem
  (m,n) = size(AtA)

  # Set up starting point and tolerance
  if isempty(x)
    x = AtA\Atb
  end
  x = max.(x, 0.0)
  tol = 100*eps(eltype(Atb))
  maxiter = 1000

  # Allocate storage for step, residual, gradient/multipliers
  p = zeros(n)
  r = zeros(n)
  rr = zeros(n)

  # Set initial passive set
  P = vec(x .> 0.0)

  # Main loop
  for i = 1:maxiter

    # Constrained Newton direction
    fill!(p,0.)
    rr[:] = AtA*x-Atb
    p[P] = AtA[P,P]\rr[P]

    if norm(p, Inf) < tol

      # Find constraint that should not be active (most negative multiplier)
      m = 0
      rrmin = 0.
      for j = 1:n
        if !P[j] && rr[j] < rrmin
          rrmin = rr[j]
          m = j
        end
      end

      # Release constraint or return
      if rrmin < 0.
        P[m] = true
      else
        return x
      end

    else

      # Determine step length
      alpha = 1.
      m = 0
      for j = 1:n
        if p[j] > 0. && x[j] < alpha*p[j]
          m = j
          alpha = min(alpha, x[j]/p[j])
        end
      end
      
      # Take a Newton step and adjust P
      if alpha == 1.
        x[:] -= p
      else
        x[:] -= alpha*p
        P[m] = false
      end
      x[:] = max.(x, 0.)

    end

  end  
  println("Did not converge")
  x
end

In [ ]:
Atest = rand(10,3)
btest = Atest * [1.0 1.0 -1.0]'
xtest = nnls(Atest'*Atest, Atest'*btest)
rtest = Atest'*(Atest*xtest-btest)
println("x = $xtest")
println("r = $rtest")

#### The alternating least squares algorithm (3 points)

Per our discussion in class, one step of alternating least squares for NMF involves two pieces:

1. Assuming $W$ is correct, solve a series of non-negative least squares problems (one per column of $H$ or $A$) to fill in $H$.
2. Assuming $H$ is correct, solve a series of non-negative least squares problems (one per row of $W$ or $A$) to fill in $W$.

Implement this ALS iteration, and again plot convergence.  Note that using the previous iterate for starting guesses for the columns of $H$ or rows of $W$ will probably speed up the NNLS solver.

## Tensor manipulations and the higher-order SVD (HOSVD)

We can represent a dense order 3 tensor in Julia as a simple three-way array.  Let's create a simple example with a known CP decomposition.

In [ ]:
F = rand(10,3)
G = rand(10,3)
H = rand(10,3)
A = [sum(F[i,:].*G[j,:].*H[k,:]) for i=1:10, j=1:10, k=1:10]

Now, we construct the flattenings in each mode.

In [ ]:
A1 = zeros(10,100)
A2 = zeros(10,100)
A3 = zeros(10,100)

for i = 1:10
    A1[i,:] = reshape(A[i,:,:], 100)
    A2[i,:] = reshape(A[:,i,:], 100)
    A3[i,:] = reshape(A[:,:,i], 100)
end

The building block for the HOSVD is the orthogonal factors from SVDs of the various flattenings.

In [ ]:
U1, S1, V1 = svd(A1)
U2, S2, V2 = svd(A2)
U3, S3, V3 = svd(A3)

In order to make sense of the SVD, we need to consider a linear transformation in each mode, i.e. a transformation of the form
$$
  C_{ijk} = \sum_{p, q, r} A_{pqr} X_{ip} Y_{jq} Z_{kr}
$$

In [ ]:
function product3(A, X, Y, Z)
    C = zeros(size(X)[1], size(Y)[1], size(Z)[1])
    for p = 1:size(X)[2]
        for q = 1:size(Y)[2]
            for r = 1:size(Z)[2]
                C += [A[p,q,r]*X[i,p]*Y[j,q]*Z[k,r] for i=1:size(X)[1], j=1:size(Y)[1], k=1:size(Z)[1]]
            end
        end
    end
    return C
end

The higher-order SVD of $A$ now looks like
$$
  A = S \times_1 U \times_2 V \times_3 W,
$$
where $U, V, W$ are orthogonal matrices.  In componenent form, this is
$$
  A_{ijk} = \sum_{p,q,r} S_{pqr} U_{ip} V_{jq} W_{kr}.
$$
We can recover $S$ by transposing the orthogonal factors, i.e.
$$
  S = A \times_1 U^T \times_2 V^T \times_3 W^T,
$$
or in component form
$$
  S_{pqr} = \sum_{i,j,k} A_{ijk} U_{ip} V_{jq} W_{kr}.
$$

In [ ]:
S = product3(A, U1', U2', U3')

In this case, the $S$ tensor is all zero (save for roundoff) outside of a 3-by-3-by-3 leading subtensor (this is a case where we have *multilinear rank* of $(3,3,3)$).  So we are able to recover a compressed representation of the full tensor in terms of the leading part of the higher-order SVD.

In [ ]:
U1k = U1[:,1:3]
U2k = U2[:,1:3]
U3k = U3[:,1:3]
Sk = product3(A, U1k', U2k', U3k')

In [ ]:
norm(A - product3(Sk, U1k, U2k, U3k))

However, while truncating the higher-order SVD recovers the tensor exactly in this case, in general we do not have the analogue of the Eckart-Young theorem: truncating the higher-order SVD does *not* generally give the best Frobenius norm approximation of a given multilinear rank.

## Alternating least squares for Tucker

The truncated HOSVD does not give the best approximation with a given multilinear rank.  Finding the best approximation is the *Tucker* approximation problem.  It turns out that we can get rid of the core tensor $S$ in the problem
$$
  \mbox{minimize } \|A-S \times_1 U \times_2 V \times_3 W\|_F^2
$$
where $U, V, W$ are rectangular matrices with orthonormal columns; the problem is equivalent to
$$
  \mbox{maximize } \|S\|_F^2 \mbox{ where } S = A \times_1 U^T \times_2 V^T \times_3 W^T.
$$
The *alternating least squares* algorithm for the Tucker decomposition updates $U$ assuming that $V$ and $W$ are correct, then updates $V$, then updates $W$.  In each case, we can compute the updated matrix solving the maximization problem by computing an SVD of a flattening; that is, if we let $g_k(B)$ refer to the first $k$ columns of the $U$ matrix in the SVD of $B$, we have
$$\begin{align*}
  U &:= g_k( (A \times_1 I \times_2 V^T \times_3 W^T)_{(1)}) \\
  V &:= g_k( (A \times_1 U^T \times_2 I \times_3 W^T)_{(2)}) \\
  W &:= g_k( (A \times_2 U^T \times_2 V^T \times_3 I)_{(3)})
\end{align*}$$
where the parenthesized subscript indicates the modes that are the columns in the flattening.
The "modal products, flatten, then SVD" updates can also be rewritten as "flatten, multiply by a Kronecker product, then SVD."  Needless to say, it always takes me some mumbling over indices to get this right.

Let's use this approach to demonstrate for the case of a Tucker decomposition of multilinear rank $(2,2,2)$.  The norm of the truncated singular value matrix at each step should be monotonically increasing.

In [ ]:
# Initialize with the orthogonal factors from the HOSVD
Ut1 = U1[:,1:2]
Ut2 = U2[:,1:2]
Ut3 = U3[:,1:2]

# Compute an updated matrix factor
function update_U(B, k)
    UU, SS, VV = svd(B)
    return UU[:,1:k], norm(SS[1:k])
end

# Track the norm of the S factor over iterations
Snorm = zeros(10)
Snorm[1] = norm(S[1:2,1:2,1:2])
for k = 2:10
    Ut1, ϕ = update_U(A1 * kron(Ut3, Ut2), 2)
    Ut2, ϕ = update_U(A2 * kron(Ut3, Ut1), 2)
    Ut3, ϕ = update_U(A3 * kron(Ut2, Ut1), 2)
    Snorm[k] = ϕ
end

# Plot the norm of the S matrix
plot(Snorm)

Plotting the differences between $S$ norms at consecutive steps gives us a hint of fast linear convergence.

In [ ]:
plot(Snorm[2:end]-Snorm[1:end-1], yscale=:log10)

## Alternating least squares for CP

The CP decomposition is a sum-of-rank-one type of decomposition that does not insist on orthonormal factors, i.e. for the third-order case we have
$$
  A = \sum_{j=1}^r \lambda_j F_{:,j} \circ G_{:,j} \circ H_{:,j}.
$$
Here, too, the standard approach involves alternating least squares problems, followed by normalization of the columns:
$$\begin{align*}
  \mbox{minimize } & \|A_{(1)} - \tilde{F} (H \odot G)^T\|^2, & \lambda_{j} F_{:,j} & = \tilde{F}_{:,j} \\
  \mbox{minimize } & \|A_{(2)} - \tilde{G} (H \odot F)^T\|^2, & \lambda_{j} G_{:,j} & = \tilde{G}_{:,j} \\
  \mbox{minimize } & \|A_{(3)} - \tilde{H} (G \odot F)^T\|^2, & \lambda_{j} H_{:,j} & = \tilde{H}_{:,j}
\end{align*}$$
Recall that $\odot$ refers to the Khatri-Rao product, which is composed of Kronecker products of matching columns.  This is therefore a very tall skinny least squares problem.  There is some special structure that one can take advantage of, but we are willing to be lazy and formulate it in the usual way.

We begin with functions to form the Khatri-Rao product of two matrices, to do column normalization, and to form the tensor represented by a given set of scale factors and matrices.

In [ ]:
function krprod(B, C)
    mB, nB = size(B)
    mC, nC = size(C)
    BC = zeros(mB*mC, nB)
    for j = 1:nB
        BC[:,j] = kron(B[:,j], C[:,j])
    end
    return BC
end

function colnormalize(Ft)
    mF, nF = size(Ft)
    λs = zeros(nF)
    F = zeros(mF, nF)
    for j = 1:nF
        λs[j] = norm(Ft[:,j])
        F[:,j] = Ft[:,j] / λs[j]
    end
    return F, λs
end

function form_cp(λs, F, G, H)
    A = zeros(size(F)[1], size(G)[1], size(H)[1])
    for p = 1:length(λs)
        for i = 1:size(F)[1]
            for j = 1:size(G)[1]
                for k = 1:size(H)[1]
                    A[i,j,k] += λs[p] * F[i,p] * G[j,p] * H[k,p]
                end
            end
        end
    end
    return A
end

### The alternating least squares algorithm (3 points)

Complete the following loop for the alternating least squares algorithm for the CP decomposition, and plot the residual error on a semi-log scale as above.

In [ ]:
# Set up a good (but not exact) initial guess
Fc, λsF = colnormalize(F)
Gc, λsG = colnormalize(G)
Hc, λsH = colnormalize(H)
Fc += 0.1 * rand(10,3)
Gc += 0.1 * rand(10,3)
Hc += 0.1 * rand(10,3)
λs = λsF .* λsG .* λsH

resids = zeros(500)
resids[1] = norm(A-form_cp(λs, Fc, Gc, Hc))
for k = 2:500
    # TODO: Fill in with ALS step
    resids[k] = norm(A-form_cp(λs, Fc, Gc, Hc))
end
# TODO: Plot residuals

### Khatri-Rao and Gram matrices (3 points)

The Gram matrix that appears in the normal equations for a least squares problem involving a Khatri-Rao product (i.e. minimizing $\|(B \odot C) u - d\|^2$) has a special form:
$$
  (B \odot C)^T (B \odot C) = (B^T B) .* (C^T C)
$$
where $.*$ refers to the Hadamard (elementwise) product.  Explain why this is true.